In [27]:
import pandas as pd
df = pd.read_csv('/Users/hakukazuho/Desktop/Luxurious-Hotel-Review/cleaned-hotel-review-for-ba820/hotel_reviews_cleaned.csv')

df = df.iloc[:10000]

In [28]:
!pip install mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Convert 'Tags' from string representation of list to actual list
df['Tags'] = df['Tags'].str.strip("[]").str.replace("'", "").str.split(', ')

In [29]:
import nltk

# Function to extract keywords/tags from a positive review
def extract_keywords(review):
    tokens = nltk.word_tokenize(review)
    tagged_tokens = nltk.pos_tag(tokens)
    
    keywords = []
    noun = None
    for word, pos in tagged_tokens:
        if pos.startswith('NN'):  # Look for nouns
            noun = word
        elif pos.startswith('JJ') and noun:  # Look for adjectives describing nouns
            keywords.append(f"{word} {noun}")
            noun = None
    
    return keywords

df['Keywords'] = df['Positive_Review'].apply(extract_keywords)

df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,Longitude,Latitude,Keywords
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[ Leisure trip , Couple , Duplex Double Room...",0 days,4.915968,52.360576,[beautiful hotel]
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[ Leisure trip , Couple , Duplex Double Room...",0 days,4.915968,52.360576,"[great hotel, original offer, new payment, lac..."
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[ Leisure trip , Family with young children ,...",3 days,4.915968,52.360576,"[good Location, ok staff, nice range]"
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[ Leisure trip , Solo traveler , Duplex Doub...",3 days,4.915968,52.360576,"[nice location, nice restaurant]"
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[ Leisure trip , Couple , Suite , Stayed 2 ...",10 days,4.915968,52.360576,[Romantic building]


In [30]:
# Function to extract the noun from a keyword combo of adjective + noun
def extract_noun(keyword):
    tokens = nltk.word_tokenize(keyword)
    tagged_tokens = nltk.pos_tag(tokens)
    nouns = [word for word, pos in tagged_tokens if pos.startswith('NN')]
    return nouns[0] if nouns else None


df['text_tags'] = df['Keywords'].apply(lambda keywords: list(set([extract_noun(keyword) for keyword in keywords])))
df['text_tags'] = df['text_tags'].apply(lambda tags: [tag.capitalize() if tag is not None else None for tag in tags])
df.head()


,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,Longitude,Latitude,Keywords,text_tags
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[ Leisure trip , Couple , Duplex Double Room...",0 days,4.915968,52.360576,[beautiful hotel],[Hotel]
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[ Leisure trip , Couple , Duplex Double Room...",0 days,4.915968,52.360576,"[great hotel, original offer, new payment, lac...","[Option, Things, Wrap, Anyone, Hotel, Payment,..."
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[ Leisure trip , Family with young children ,...",3 days,4.915968,52.360576,"[good Location, ok staff, nice range]","[Range, Staff, Location]"
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[ Leisure trip , Solo traveler , Duplex Doub...",3 days,4.915968,52.360576,"[nice location, nice restaurant]","[Location, Restaurant]"
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[ Leisure trip , Couple , Suite , Stayed 2 ...",10 days,4.915968,52.360576,[Romantic building],[Building]


In [31]:
df['transactions'] = df.apply(lambda row: [row['Hotel_Name']] + row['text_tags'], axis=1)
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,Longitude,Latitude,Keywords,text_tags,transactions
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[ Leisure trip , Couple , Duplex Double Room...",0 days,4.915968,52.360576,[beautiful hotel],[Hotel],"[Hotel Arena, Hotel]"
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[ Leisure trip , Couple , Duplex Double Room...",0 days,4.915968,52.360576,"[great hotel, original offer, new payment, lac...","[Option, Things, Wrap, Anyone, Hotel, Payment,...","[Hotel Arena, Option, Things, Wrap, Anyone, Ho..."
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[ Leisure trip , Family with young children ,...",3 days,4.915968,52.360576,"[good Location, ok staff, nice range]","[Range, Staff, Location]","[Hotel Arena, Range, Staff, Location]"
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[ Leisure trip , Solo traveler , Duplex Doub...",3 days,4.915968,52.360576,"[nice location, nice restaurant]","[Location, Restaurant]","[Hotel Arena, Location, Restaurant]"
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[ Leisure trip , Couple , Suite , Stayed 2 ...",10 days,4.915968,52.360576,[Romantic building],[Building],"[Hotel Arena, Building]"


In [32]:
from collections import Counter

table1 = df.groupby('Hotel_Name')['transactions'].sum().reset_index()

hotel_lists = {}

for hotel_name, transactions in table1.groupby('Hotel_Name')['transactions']:
    transaction_list = transactions.tolist()
    flattened_list = [item for sublist in transaction_list for item in sublist]
    counter = Counter(flattened_list)
    top_50_elements = [element for element, count in counter.most_common(10)]
    hotel_lists[hotel_name] = top_50_elements

In [33]:
top_tags = pd.DataFrame({
    'Hotel_Name': list(hotel_lists.keys()),
    'Top_Tags': list(hotel_lists.values())
})

merged_table = df.merge(top_tags, on='Hotel_Name')

# Check each item in the list in transactions and add to a new list if it's in the list in Top_Tags
merged_table['new_transactions'] = merged_table.apply(lambda row: [item for item in row['transactions'] if item in row['Top_Tags']], axis=1)
merged_table.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,...,Reviewer_Score,Tags,days_since_review,Longitude,Latitude,Keywords,text_tags,transactions,Top_Tags,new_transactions
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,...,2.9,"[ Leisure trip , Couple , Duplex Double Room...",0 days,4.915968,52.360576,[beautiful hotel],[Hotel],"[Hotel Arena, Hotel]","[Hotel Arena, Staff, Hotel, Room, Location, Be...","[Hotel Arena, Hotel]"
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,...,7.5,"[ Leisure trip , Couple , Duplex Double Room...",0 days,4.915968,52.360576,"[great hotel, original offer, new payment, lac...","[Option, Things, Wrap, Anyone, Hotel, Payment,...","[Hotel Arena, Option, Things, Wrap, Anyone, Ho...","[Hotel Arena, Staff, Hotel, Room, Location, Be...","[Hotel Arena, Hotel]"
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,...,7.1,"[ Leisure trip , Family with young children ,...",3 days,4.915968,52.360576,"[good Location, ok staff, nice range]","[Range, Staff, Location]","[Hotel Arena, Range, Staff, Location]","[Hotel Arena, Staff, Hotel, Room, Location, Be...","[Hotel Arena, Staff, Location]"
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,...,3.8,"[ Leisure trip , Solo traveler , Duplex Doub...",3 days,4.915968,52.360576,"[nice location, nice restaurant]","[Location, Restaurant]","[Hotel Arena, Location, Restaurant]","[Hotel Arena, Staff, Hotel, Room, Location, Be...","[Hotel Arena, Location]"
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,...,6.7,"[ Leisure trip , Couple , Suite , Stayed 2 ...",10 days,4.915968,52.360576,[Romantic building],[Building],"[Hotel Arena, Building]","[Hotel Arena, Staff, Hotel, Room, Location, Be...","[Hotel Arena, Building]"


In [39]:

from mlxtend.frequent_patterns import apriori, association_rules

# apply transactionencoder
te = TransactionEncoder()
merged_table['new_transactions'] = merged_table['new_transactions'].apply(lambda x: [item for item in x if item is not None])
te_ary = te.fit(merged_table['new_transactions']).transform(merged_table['new_transactions'])
df_te = pd.DataFrame(te_ary, columns=te.columns_)

# create a unique list of hotel name
antecedents_hotel = merged_table['Hotel_Name'].unique()

# apply Apriori algorithm
frequent_itemsets = apriori(df_te, min_support=0.0000001, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
rules['antecedents'] = rules['antecedents'].apply(lambda x: set(x))
rules['consequents'] = rules['consequents'].apply(lambda x: set(x))

# filter rules
single_antecedent_consequent_rules = rules[(rules['antecedents'].map(len) == 1) &
                                           (rules['consequents'].map(len) == 1) &
                                           (rules['antecedents'].apply(lambda x: list(x)[0]).isin(antecedents_hotel))]

rules_table = single_antecedent_consequent_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

In [40]:
rules_table

,antecedents,consequents,support,confidence,lift
3,{Apex Temple Court Hotel},{Hotel},0.0137,0.132112,1.412961
4,{Apex Temple Court Hotel},{Location},0.0130,0.125362,0.941153
6,{Apex Temple Court Hotel},{Room},0.0170,0.163934,1.285760
9,{Apex Temple Court Hotel},{Staff},0.0240,0.231437,1.092714
30,{Kube Hotel Ice Bar},{Bed},0.0002,0.105263,2.899812
55,{K K Hotel George},{Breakfast},0.0070,0.123675,2.147134
102,{Grand Royale London Hyde Park},{Hotel},0.0190,0.111830,1.196048
104,{Grand Royale London Hyde Park},{Location},0.0275,0.161860,1.215165
106,{Grand Royale London Hyde Park},{Room},0.0197,0.115951,0.909416
109,{Grand Royale London Hyde Park},{Staff},0.0441,0.259564,1.225517


In [48]:
def popular_tags(hotel_name):
  print(rules_table[rules_table['antecedents'] == {hotel_name}]['consequents'])

In [49]:
popular_tags('Hotel Arena')

111    {Hotel}
141     {Room}
142    {Staff}
Name: consequents, dtype: object
